In [1]:
# -*- coding: utf-8 -*-
"""DL_Proj_Sep_Classes.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1ttl3H0xXe6ujhuUwA7gugqgaAGYxwUzZ
"""

# Commented out IPython magic to ensure Python compatibility.
import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial
import PIL
import PIL.Image

# %tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

np.random.seed(42) # note that you must use the same seed to ensure consistentcy in your training/validation/testing
tf.random.set_seed(42)

import pandas as pd
import numpy as np

In [2]:
# each batch in train_ds or validation_ds tensor: 32 image tensors (224 x 224 x 3); 1D tensor with 32 class labels

from sklearn.datasets import load_files 
from keras.utils import np_utils

from keras.preprocessing import image
from tqdm import tqdm # progress bar

data_dir = "/home/cew4pf/dl_project/content/test_dir/"
batch_size = 32;
# IMPORTANT: Depends on what pre-trained model you choose, you will need to change these dimensions accordingly --> change to 224 since expected size for MobileNet
img_height = 224; 
img_width = 224;

# Training Dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    image_size= (img_height, img_width),
    batch_size = batch_size
)

# Validation Dataset
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

Found 12370 files belonging to 39 classes.
Using 9896 files for training.


2022-04-25 12:25:51.396028: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-25 12:25:51.396082: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (udc-aw34-10c0): /proc/driver/nvidia/version does not exist
2022-04-25 12:25:51.398875: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 12370 files belonging to 39 classes.
Using 2474 files for validation.


### **Read in Pre-Trained Models**

In [3]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

#### **MobileNet**

In [4]:
mn_model = tf.keras.models.load_model('/home/cew4pf/dl_project/tmp/mn_model')

In [5]:
mn_model._name = 'mn_model'

In [6]:
mn_model.summary()

Model: "mn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                          

In [7]:
mn_model.evaluate(validation_ds)

78/78 [==============================] - 19s 228ms/step - loss: 1.4334 - accuracy: 0.7352


[1.433426022529602, 0.7352465391159058]

#### **InceptionNet**

In [8]:
inc_model = tf.keras.models.load_model('/home/cew4pf/dl_project/tmp/inc_model')

In [9]:
inc_model._name = 'inc_model'

In [10]:
inc_model.summary()

Model: "inc_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0 

In [11]:
inc_model.evaluate(validation_ds)

78/78 [==============================] - 18s 212ms/step - loss: 0.9922 - accuracy: 0.7017


[0.9921690225601196, 0.7016976475715637]

#### **VGG16**

In [12]:
vgg_model = tf.keras.models.load_model('/home/cew4pf/dl_project/tmp/vgg_model')

In [13]:
vgg_model._name = 'vgg_model'

In [14]:
vgg_model.summary()

Model: "vgg_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                  

In [15]:
vgg_model.evaluate(validation_ds)

78/78 [==============================] - 57s 725ms/step - loss: 1.3468 - accuracy: 0.5982


[1.3468308448791504, 0.598221480846405]

#### **ResNet**

In [16]:
res_model = tf.keras.models.load_model('/home/cew4pf/dl_project/tmp/res_model')

In [17]:
res_model._name = 'res_model'

In [18]:
res_model.summary()

Model: "res_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0 

In [19]:
res_model.evaluate(validation_ds)

78/78 [==============================] - 34s 418ms/step - loss: 1.3820 - accuracy: 0.7061


[1.382044792175293, 0.7061439156532288]

#### **EfficientNet**

In [20]:
eff_model = tf.keras.models.load_model('/home/cew4pf/dl_project/tmp/eff_model')

In [21]:
eff_model._name = 'eff_model'

In [22]:
eff_model.summary()

Model: "eff_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 efficientnetv2-b3 (Function  (None, 7, 7, 1536)       12930622  
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1536)              0         
                                                                 
 dense (Dense)               (None, 39)                59

In [23]:
eff_model.evaluate(validation_ds)

78/78 [==============================] - 27s 321ms/step - loss: 2.0642 - accuracy: 0.7676


[2.0642387866973877, 0.7675828337669373]

### **Inputs and Outputs**

In [24]:
inputs = tf.keras.Input(shape = (224, 224, 3))
outputs = tf.keras.layers.Dense(39, activation = "softmax")(inputs)

### **Feed inputs (y values) to each model to get predictions**

In [25]:
models = [mn_model, inc_model, vgg_model, res_model, eff_model]

n_models = len(models)

In [26]:
preds = [model(inputs) for model in models]
# y_1 = mn_model(inputs)
# y_2 = inc_model(inputs)
# y_3 = vgg_model(inputs)
# y_4 = res_model(inputs)
# y_5 = eff_model(inputs)

### **Outputs the average of all model predictions**

In [27]:
outputs = keras.layers.average(preds)

### **Create and compile ensemble model**

In [28]:
keras.backend.clear_session()

In [29]:
ensemble_model = keras.Model(inputs=inputs, outputs=outputs)

In [30]:
base_learning_rate = 0.001

ensemble_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = base_learning_rate),
                                                   loss = "sparse_categorical_crossentropy",
                                                   metrics = ['accuracy'])

In [31]:
ensemble_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 mn_model (Functional)          (None, 39)           2307943     ['input_1[0][0]']                
                                                                                                  
 inc_model (Functional)         (None, 39)           21882695    ['input_1[0][0]']                
                                                                                                  
 vgg_model (Functional)         (None, 39)           14734695    ['input_1[0][0]']            

### **Evaluate ensemble model**

In [32]:
ensemble_model.evaluate(validation_ds)

78/78 [==============================] - 128s 2s/step - loss: 0.7606 - accuracy: 0.7652


[0.7605798244476318, 0.7651576399803162]

### **Tuned Weights**
* **(based on testing different weight values with `dl_project_ensemble.py`)**

In [33]:
tuned_weights = [1/9, 1/3, 1/9, 1/9, 1/3]

In [34]:
tuned_outputs = keras.layers.average([pred * weight for pred, weight in zip(preds, tuned_weights)])

In [35]:
keras.backend.clear_session()

In [36]:
tuned_ensemble_model = keras.Model(inputs=inputs, outputs=tuned_outputs)

In [37]:
base_learning_rate = 0.001

tuned_ensemble_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = base_learning_rate),
                                                   loss = "sparse_categorical_crossentropy",
                                                   metrics = ['accuracy'])

In [38]:
tuned_ensemble_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 mn_model (Functional)          (None, 39)           2307943     ['input_1[0][0]']                
                                                                                                  
 inc_model (Functional)         (None, 39)           21882695    ['input_1[0][0]']                
                                                                                                  
 vgg_model (Functional)         (None, 39)           14734695    ['input_1[0][0]']            

In [39]:
tuned_ensemble_model.evaluate(validation_ds)

78/78 [==============================] - 128s 2s/step - loss: 0.7450 - accuracy: 0.7769


[0.7449920773506165, 0.7768795490264893]

### **Test with Higher Weights**

In [40]:
test_weights = [1/81, 39/81, 1/81, 1/81, 39/81]

In [41]:
test_outputs = keras.layers.average([pred * weight for pred, weight in zip(preds, test_weights)])

In [42]:
keras.backend.clear_session()

In [43]:
test_ensemble_model = keras.Model(inputs=inputs, outputs=test_outputs)

In [44]:
base_learning_rate = 0.001

test_ensemble_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = base_learning_rate),
                                                   loss = "sparse_categorical_crossentropy",
                                                   metrics = ['accuracy'])

In [45]:
test_ensemble_model.evaluate(validation_ds)

78/78 [==============================] - 127s 2s/step - loss: 0.7726 - accuracy: 0.7773


[0.7725872993469238, 0.7772837281227112]

### **Test with Even Higher Weights**

In [46]:
test_weights = [1/243, 120/243, 1/243, 1/243, 120/243]

In [47]:
test_outputs = keras.layers.average([pred * weight for pred, weight in zip(preds, test_weights)])

In [48]:
keras.backend.clear_session()

In [49]:
test_ensemble_model = keras.Model(inputs=inputs, outputs=test_outputs)

In [50]:
base_learning_rate = 0.001

test_ensemble_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = base_learning_rate),
                                                   loss = "sparse_categorical_crossentropy",
                                                   metrics = ['accuracy'])

In [51]:
test_ensemble_model.evaluate(validation_ds)

78/78 [==============================] - 130s 2s/step - loss: 0.7844 - accuracy: 0.7765


[0.784392237663269, 0.7764753699302673]

### **Test with Select Models**

In [76]:
test_weights = [0, 1/2, 0, 0, 1/2]

In [77]:
test_outputs = keras.layers.average([pred * weight for pred, weight in zip(preds, test_weights)])

In [78]:
keras.backend.clear_session()

In [79]:
test_ensemble_model = keras.Model(inputs=inputs, outputs=test_outputs)

In [80]:
base_learning_rate = 0.001

test_ensemble_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = base_learning_rate),
                                                   loss = "sparse_categorical_crossentropy",
                                                   metrics = ['accuracy'])

In [81]:
test_ensemble_model.evaluate(validation_ds)

78/78 [==============================] - 128s 2s/step - loss: 0.8030 - accuracy: 0.7765


[0.8030121922492981, 0.7764753699302673]

### **Test without Worst Performing Model (VGG)**

In [64]:
test_weights = [1/6, 1/3, 0, 1/6, 1/3]

In [65]:
test_outputs = keras.layers.average([pred * weight for pred, weight in zip(preds, test_weights)])

In [66]:
keras.backend.clear_session()

In [67]:
test_ensemble_model = keras.Model(inputs=inputs, outputs=test_outputs)

In [68]:
base_learning_rate = 0.001

test_ensemble_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = base_learning_rate),
                                                   loss = "sparse_categorical_crossentropy",
                                                   metrics = ['accuracy'])

In [69]:
test_ensemble_model.evaluate(validation_ds)

78/78 [==============================] - 129s 2s/step - loss: 0.7536 - accuracy: 0.7757


[0.753592848777771, 0.7756669521331787]

### **Sources**

* Creating an ensemble model from pre-trained models: https://www.tensorflow.org/guide/keras/functional
* Renaming a model with `model._name = "NAME"` to prevent errors about models being named the same thing in ensemble model: https://stackoverflow.com/questions/56886442/error-when-trying-to-rename-a-pretrained-model-on-tf-keras